In [1]:
import pandas as pd

In [ ]:
!pip install mysql-connector-python

In [4]:
data = pd.read_csv('ready_to_ingest_data.csv')

In [5]:
data.shape

(638, 21)

In [6]:
data.head()

,ticker,name,currency,exchange,industry,environment_grade,environment_level,social_grade,social_level,governance_grade,...,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik,annual_eps,gross_profit
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,A,High,BB,Medium,BB,...,510,316,321,1147,2022-04-19,BBB,High,1744489,1.72,2.832100e+10
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",OTHERS,A,High,BB,Medium,B,...,510,303,255,1068,2022-04-17,BBB,High,1467858,6.13,2.098100e+10
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",OTHERS,B,Medium,BB,Medium,B,...,255,385,240,880,2022-04-19,BB,Medium,277135,30.06,5.849000e+09
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,A,High,B,Medium,BB,...,570,298,303,1171,2022-04-18,BBB,High,851968,0.39,2.943400e+09
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BBB,High,BB,Medium,B,...,492,310,250,1052,2022-04-18,BBB,High,1335258,0.64,4.343730e+09


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ticker                638 non-null    object 
 1   name                  638 non-null    object 
 2   currency              638 non-null    object 
 3   exchange              638 non-null    object 
 4   industry              638 non-null    object 
 5   environment_grade     638 non-null    object 
 6   environment_level     638 non-null    object 
 7   social_grade          638 non-null    object 
 8   social_level          638 non-null    object 
 9   governance_grade      638 non-null    object 
 10  governance_level      638 non-null    object 
 11  environment_score     638 non-null    int64  
 12  social_score          638 non-null    int64  
 13  governance_score      638 non-null    int64  
 14  total_score           638 non-null    int64  
 15  last_processing_date  6

In [15]:
data[(data['annual_eps']!=0.0) & (data['gross_profit']!=0.0)].shape

(536, 21)

In [16]:
data = data[(data['annual_eps']!=0.0) & (data['gross_profit']!=0.0)]

In [17]:
data.columns

Index(['ticker', 'name', 'currency', 'exchange', 'industry',
       'environment_grade', 'environment_level', 'social_grade',
       'social_level', 'governance_grade', 'governance_level',
       'environment_score', 'social_score', 'governance_score', 'total_score',
       'last_processing_date', 'total_grade', 'total_level', 'cik',
       'annual_eps', 'gross_profit'],
      dtype='object')

In [18]:
import pandas as pd
import mysql.connector

df = data.copy()

# MySQL connection details
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Ravi12nov!",
    database="ESG_PowerBI"
)

cursor = mydb.cursor()

# Table creation SQL (adjust data types as needed)
create_table_query = """
CREATE TABLE IF NOT EXISTS ESG_data (
    ticker VARCHAR(255),
    name VARCHAR(255),
    currency VARCHAR(255),
    exchange VARCHAR(255),
    industry VARCHAR(255),
    environment_grade VARCHAR(255),
    environment_level VARCHAR(255),
    social_grade VARCHAR(255),
    social_level VARCHAR(255),
    governance_grade VARCHAR(255),
    governance_level VARCHAR(255),
    environment_score INT,
    social_score INT,
    governance_score INT,
    total_score INT,
    last_processing_date DATE,
    total_grade VARCHAR(255),
    total_level VARCHAR(255),
    cik INT,
    annual_eps FLOAT,
    gross_profit DOUBLE
)
"""
cursor.execute(create_table_query)
mydb.commit()

# Prepare data for insertion
data_tuples = [tuple(x) for x in df.to_numpy()]

# Insert data
insert_query = "INSERT INTO ESG_data (ticker, name, currency, exchange, industry,\
       environment_grade, environment_level, social_grade,\
       social_level,governance_grade, governance_level,\
       environment_score, social_score, governance_score, total_score,\
       last_processing_date, total_grade, total_level, cik,\
       annual_eps, gross_profit) VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s)"
cursor.executemany(insert_query, data_tuples)
mydb.commit()


# Close connection
cursor.close()
mydb.close()